In [1]:
from selenium.webdriver.edge.service import Service
def start_web():
    #登陆该网站
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    
    # 启动浏览器
    webdriver_path = 'msedgedriver_123.exe'
    service = Service(executable_path=webdriver_path)
    driver = webdriver.Edge(service = service)
    url = 'https://policy.ckcest.cn/login'
    t = driver.get('https://policy.ckcest.cn/login')
    
    #隐式等待10s
    driver.implicitly_wait(10)
    
    
    # 获取账号、密码和验证码图片的元素
    username_input = driver.find_element(By.ID,'username')
    password_input = driver.find_element(By.ID,'password')
    captcha_img = driver.find_element(By.ID,'imgVcode')
    text_input = driver.find_element(By.ID,'imageverifycode')
    
     
    # 获取验证码图片URL
    captcha_img_url = captcha_img.get_attribute('src')
    #print('验证码图片URL:', captcha_img_url)
    
    import base64
    
    # 从Base64编码的数据中提取图片内容
    base64_data = captcha_img_url[23:]
    #print(base64_data)
    image_data = base64.b64decode(base64_data)
    
    # 将图片内容保存为图片文件
    with open('captcha_image.jpg', 'wb') as f:
        f.write(image_data)
    
    #print('验证码图片已保存为captcha_image.jpg文件')
    
    #识别验证码
    
    def get_file_content(filePath):
          with open(filePath, "rb") as fp:
             return fp.read()
    
    image = get_file_content('captcha_image.jpg')
    #百度 0M0xY1SZ2evc6fh13rS0Z2WE  7lpqROeY6AqKAbArx8j5uLZ8i3owYobe
    
    from aip import AipOcr
    
    #你的 APPID AK SK 
    APP_ID = '58266535'
    API_KEY = '0M0xY1SZ2evc6fh13rS0Z2WE'
    SECRET_KEY = '7lpqROeY6AqKAbArx8j5uLZ8i3owYobe'
    
    client = AipOcr(APP_ID, API_KEY, SECRET_KEY)
    res_image = client.basicGeneral(image)
    text = res_image['words_result'][0]['words']
    print(text)
    
    # 输入账号、密码和验证码
    username_input.send_keys('qweasdzxcrtyfghvbn')
    password_input.send_keys('?5201314Dyf')
    text_input.send_keys(text)
    
    #点击登陆按钮，有时会出现没有成功识别的情况
    submit_button = driver.find_element(By.ID,'submitid').click()
    
    return driver

In [2]:
#爬取单个页面
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


#验证页面链接是否对应的是该文件
def open_passage_by_row(start, end):
    
    # 读取xlsx文件
    xlsx_path = '地方汇总-29190条.xlsx'
    df = pd.read_excel(xlsx_path)
    driver = start_web()
    # 遍历每一行
    for index, row in df.iterrows():
        # 获取标题和URL字段的值
        id = row['地方编号']
        title = row['标题']
        url = row['标题链接']
        if int(id) >= start and int(id) <= end:
            # 打印或处理标题和URL
            print(f"ID:{id}, 标题: {title}, URL: {url}")
            try:
                driver.get(url)
                #获取文章标题等字段写入文件
                open_title = driver.find_element(By.XPATH,'//*[@id="gridleft"]/div/div[1]/h2')
                get_important_element(id, title, url, driver)
                result = paqukuai(driver)
                xieru_csv(id, title, result)
                print("------------------------------------------------------------------")
            except TimeoutException or WebDriverException:
                open_passage_by_row(int(id), end)
    print("已完成爬取！")
                
    
#判断网页中存在的块
def paqukuai(driver):
    #初始化字典
    yinyong_dict = {}
    beiyinyong_dict = {}
    xiangsi_dict = {}
    
    #对法宝联想下面的块进行爬取
    try:
        # 获得相关信息有多少个
        parent_element = driver.find_element(By.XPATH, '//*[@id="pkulaw-association"]')        
        # 在这个parent_element中查找所有div标签
        inside_element = parent_element.find_elements(By.CLASS_NAME,'lenovo')        
        # 计算找到的div标签数量
        number_of_divs = len(inside_element)
        
        for i in range(number_of_divs):
            div_element = inside_element[i]
            #print(div_element)
            head_of_div = div_element.find_element(By.TAG_NAME, 'h4').text
            #print(str(head_of_div)[:4])
            if str(head_of_div)[:4] == "本篇引用":
                print("本篇引用的法规:")
                yinyong_dict = BenPianYinYong(div_element)
                print(yinyong_dict)
            if str(head_of_div)[:4] == "引用本篇":
                print("引用本篇的法规 案例 论文:")
                beiyinyong_dict = BenPianYinYong(div_element)
                print(beiyinyong_dict)
                
    except NoSuchElementException:
        print("没有法宝联想")
    
    #对智能发现下面的块进行爬取
    try:
        # 获得相关信息有多少个
        parent_element = driver.find_element(By.XPATH, '//*[@id="rightZhfx"]')         
        # 在这个parent_element中查找所有div标签
        inside_element = parent_element.find_elements(By.CLASS_NAME,'lenovo')        
        # 计算找到的div标签数量
        number_of_divs = len(inside_element)        
        #print("找到的智能发现div标签数量:", number_of_divs)        
        for i in range(number_of_divs):
            div_element = inside_element[i]
            head_of_div = div_element.find_element(By.TAG_NAME, 'h4').text
            if head_of_div == "本篇相似法规发现":
                xiangsi_dict = BenPianYinYong(div_element)
                print("本篇相似法规发现:")
                print(xiangsi_dict)
        
    except NoSuchElementException:
        print("没有智能发现")
        
    return yinyong_dict, beiyinyong_dict, xiangsi_dict

#判断有无“更多”，有则点击进入，无则不进入。或者不爬更多也行 


#爬取块并存入字典
def BenPianYinYong(div_element):
    # 初始化一个空字典来存储法规名称和链接
    laws_dict = {}   
    # 使用Selenium查找所有的<a>标签
    a_tags = div_element.find_elements(By.TAG_NAME,'a')    
    for a_tag in a_tags:
        # 获取法规名称
        law_name = a_tag.text       
        # 获取法规对应的链接
        law_link = a_tag.get_attribute('href')            
        # 存入字典
        laws_dict[law_name] = law_link   
    #print(laws_dict)
    return laws_dict

#写入引用的csv文件
def xieru_csv(id, title, result):
    yinyong_dict = result[0]
    beiyinyong_dict = result[1]
    xiangsi_dict = result[2]
    rows = []
    # 准备要写入的数据
    headers = ['政策A_id', '政策A_标题', '关系', '政策B_标题', '政策B的链接']
    if len(yinyong_dict) != 0:
        for key, value in yinyong_dict.items():
            relation = (id, title, '引用', key, value)
            rows.append(relation)
    if len(beiyinyong_dict) != 0:
        for key, value in beiyinyong_dict.items():
            relation = (id, title, '被引用', key, value)
            rows.append(relation)
    if len(xiangsi_dict) != 0:
        for key, value in xiangsi_dict.items():
            relation = (id, title, '相似于', key, value)
            rows.append(relation)
    
    # 写入CSV文件
    with open('policies_local_10000.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)       
        # 写入标题行
        #writer.writerow(headers)        
        # 写入数据行
        for row in rows:
            writer.writerow(row)

#获取重要字段
def get_important_element(id, title, url, driver):
    def get_office(i):
        #制定机关outer_xpath
        office = []
        office_outer_element = i.find_element(By.XPATH, '..')   
        office_inner_elements = office_outer_element.find_elements(By.TAG_NAME, 'a')   
        for i in office_inner_elements:
            office_name = i.text
            if office_name != '' and office_name != '机构沿革' :
                office.append(office_name)
        return office
    
    #法宝引证码
    fabao_id = driver.find_element(By.XPATH, '//*[@id="gridleft"]/div/div[1]/div[1]/span/a').text
        
    #获取发文字号、发文日期、施行日期等一系列元素
    document_num = ''
    post_data = ''
    effective_data = ''
    category = ''
    timeliness = ''
    rank_of_service = ''
    office = []
    
    outer_element = driver.find_element(By.XPATH, '//*[@id="gridleft"]/div/div[1]/div[2]/ul')
    inner_elements = outer_element.find_elements(By.TAG_NAME, 'strong') 
    for i in inner_elements:
        parent_div_text = i.find_element(By.XPATH, "..").text
        if i.text == '发文字号：':
            document_num =  parent_div_text[6:]
        elif i.text == '公布日期：':
            post_data = parent_div_text[6:]
        elif i.text == '施行日期：':
            effective_data = parent_div_text[6:]
        elif i.text == '法规类别：':
            category = parent_div_text[6:].split(' ')
        elif i.text == '时效性：':
            timeliness = parent_div_text[5:]
        elif i.text == '效力位阶：':
            rank_of_service = parent_div_text[6:]
        elif i.text == '制定机关：':
            office = get_office(i)

    row = (id, title, url, fabao_id, office, document_num, post_data, effective_data, timeliness, rank_of_service, category)

    with open('full_element_10000.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)             
        writer.writerow(row)


In [3]:
open_passage_by_row(19500,20000)

5xfm
ID:19500, 标题: 海南省科学技术厅关于“胡椒碱基于NF—κB信号通路减轻炎症反应机制研究及应用”项目验收结果的通知, URL: https://pkulaw.com/lar/5c6b4fc093718f4569b4b2fd53b7fdaabdfb.html?way=listView
没有法宝联想
没有智能发现
------------------------------------------------------------------
ID:19501, 标题: 海南省科学技术厅关于开展2022年度海南省创新联合体认定工作的通知, URL: https://pkulaw.com/lar/dce756a75d810e82b645fc44938efe38bdfb.html?way=listView
引用本篇的法规 案例 论文:
{'海南省科学技术厅关于2022年度海南省创新联合体认定结果的通知': 'https://pkulaw.com/lar/5f75bc9976d9ab6dc411d1ae7cc9e2f0bdfb.html?way=textRightFblx'}
没有智能发现
------------------------------------------------------------------
ID:19502, 标题: 海南省高新技术企业认定委员会办公室关于开展2022年度第三批高新技术企业认定工作的通知, URL: https://pkulaw.com/lar/da0b4e9595141e859863465a586dcc69bdfb.html?way=listView
没有法宝联想
没有智能发现
------------------------------------------------------------------
ID:19503, 标题: 海南省科学技术厅关于开展2022年度第三批高新技术种子企业、瞪羚企业、领军企业认定申报工作的通知, URL: https://pkulaw.com/lar/0d95ccb250df03705b69d8014ca7e21dbdfb.html?way=listView
没有法宝联想
没有智能发现
----------------------------------------